In [1]:
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score, mean_squared_error, r2_score
import statistics
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from keras.optimizers import SGD, Adam, RMSprop, Adamax
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../df.csv")
doc2vec50 = pd.read_csv("../dataset/doc2vec_50.csv")
df_train= df[["essay_set", "essay","total_score", "word_count","Mistakes","reading_ease"]]
df_train = doc2vec50
df_train = pd.concat([df_train, df[["essay_set", "word_count","Mistakes","reading_ease"]]], axis = 1, join = "inner").drop(['Unnamed: 0'], axis = 1)
y = df["total_score"]
X = df_train

In [29]:
df = pd.read_csv("../df.csv")
doc2vec300 = pd.read_csv("../dataset/doc2vec_300.csv")
df_train= df[["essay_set", "essay","total_score", "word_count","Mistakes","reading_ease"]]
df_train = doc2vec300
df_train = pd.concat([df_train, df[["essay_set", "word_count","Mistakes","reading_ease"]]], axis = 1, join = "inner").drop(['Unnamed: 0'], axis = 1)
y = df["total_score"].values
X = df_train.values


In [3]:
def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(54, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 54], return_sequences=True))
    model.add(LSTM(54, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [73]:
# doc_emb

In [4]:
cv = KFold(n_splits = 5, shuffle = True)
results = []
r2s = []
mses = []
y_pred_list = []
count = 1

for num in range(1,9):
    essay_set = pd.read_csv("../../dataset/cleandata_set_{}.csv".format(num))
    train_corpus = essay_set['essay']
    test_corpus = train_corpus.copy()
    vocab = CountVectorizer(stop_words='english', lowercase= True).fit(train_corpus)

    # generate counts for a new set of documents
    doc_emb = vocab.transform(train_corpus)
    
    vec_size = 50
    window=2
    min_count=1
    workers=8
    epochs=40
    essays = [TaggedDocument(gensim.utils.simple_preprocess(doc), [i]) for i, doc in enumerate(train_corpus)]
    
    model = Doc2Vec(essays, vector_size=vec_size, window=window, min_count=min_count, workers=workers, epochs=epochs)
    #might not need this line
    model.train(essays, total_examples=model.corpus_count, epochs=model.epochs)
    doc = [gensim.utils.simple_preprocess(doc) for i, doc in enumerate(test_corpus)]

    doc_emb = np.zeros((len(doc), vec_size))
    for i in range(len(doc)):
        doc_emb[i,:] = model.infer_vector(doc[i])
        
    df_train = pd.DataFrame(doc_emb)
    df_train = pd.concat([df_train, df[["essay_set", "word_count","Mistakes","reading_ease"]]], axis = 1, join = "inner")
    
    y = df["total_score"]
    X = df_train
    print("\n###########Set-{}###########\n".format(num))
    count = 1
    result_for_set = []
    for traincv, testcv in cv.split(X):
        print("\n--------Fold {}--------\n".format(count))
        
        X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
        X_train = X_train.to_numpy()
        X_test = X_test.to_numpy()

        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

        lstm_model = get_model()
        lstm_model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32)


        y_pred_temp = lstm_model.predict(X_test)

        y_pred = []
        for i in y_pred_temp:
            y_pred.append(int(round(i[0])))
        y_pred_temp = np.around(y_pred_temp)
        if count == 5:
             lstm_model.save_weights('final_lstm.h5')
        y_test_new = []
        for  i in list(y_test.array):
            y_test_new.append(int(i))
        result = cohen_kappa_score(y_test_new,y_pred,weights='quadratic')
        print("Kappa Score: {}".format(result))
        result_for_set.append(result)
        count += 1
    results.append(result_for_set)

KeyError: "['Unnamed: 0'] not found in axis"

In [5]:
print(df_train)

            0         1         2         3         4         5         6   \
0     0.107725 -0.382557  1.334837 -0.180234 -0.114252  0.654670 -0.951697   
1     0.559967  1.035295  0.166422  0.516241 -0.084778  0.942859 -0.597336   
2    -0.564991  0.252604  0.235742  0.760749  1.144909  0.261184 -1.308746   
3     1.101475 -0.231672  1.340678  1.060182 -2.239716  1.058062 -0.183254   
4    -0.263135  0.525271  0.858397  0.369867 -0.826594  1.089485 -0.265835   
...        ...       ...       ...       ...       ...       ...       ...   
1778 -0.175561  0.701745  2.341178  0.783649  3.520728  0.732004 -1.378145   
1779 -0.114634 -0.071539  0.986146  1.384397 -0.938375  1.962089  0.069004   
1780 -1.829511 -0.515405  2.013804 -0.271621 -0.811980  0.712048 -1.395960   
1781 -0.337402  0.022656  1.072508  0.335590  0.377162  0.357291 -0.615945   
1782  0.757166 -0.499682  1.190086  1.425673 -0.309289  0.857285 -1.217730   

            7         8         9   ...        40        41    

In [80]:
for i in range(0,8):
    print("Essay set {}".format(i+1))
    print(np.mean(results[i]))

Essay set 1
0.7430904524409885
Essay set 2
0.6430423972736458
Essay set 3
0.608976790928228
Essay set 4
0.6923776715581466
Essay set 5
0.6636116129139091
Essay set 6
0.6727591464636252
Essay set 7
0.7504665038869414
Essay set 8
0.0019747832427356336


In [35]:
X.to_numpy()

array([[-6.53643310e-01,  1.22631383e+00,  1.49060738e+00, ...,
         3.38000000e+02,  2.00000000e+01,  7.40200000e+01],
       [ 1.78830183e+00, -5.99211633e-01, -2.10853004e+00, ...,
         4.19000000e+02,  3.30000000e+01,  6.70800000e+01],
       [-2.45267693e-02,  5.21169484e-01, -8.56822908e-01, ...,
         2.79000000e+02,  2.00000000e+01,  6.82000000e+01],
       ...,
       [ 1.19136560e+00,  9.73212183e-01, -5.47442257e-01, ...,
         8.18000000e+02,  8.20000000e+01,  6.92500000e+01],
       [ 4.18220490e-01,  1.79089689e+00,  5.47983468e-01, ...,
         5.94000000e+02,  3.30000000e+01,  7.20500000e+01],
       [-8.03048015e-02,  1.26649606e+00,  1.87341690e-01, ...,
         4.68000000e+02,  1.10000000e+01,  7.20500000e+01]])

In [55]:
y = df.loc[df["essay_set"] == j]["total_score"]
X = df_train.loc[df_train["essay_set"] == j]


In [56]:
y

0        8.0
1        9.0
2        7.0
3       10.0
4        8.0
        ... 
1778     8.0
1779     7.0
1780     8.0
1781     2.0
1782     7.0
Name: total_score, Length: 1783, dtype: float64

In [57]:
X

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,essay_set,word_count,Mistakes,reading_ease
0,-0.653643,1.226314,1.490607,-0.273509,-0.126667,1.948407,-0.800371,-0.520892,-1.989100,0.305569,...,1.436761,1.510649,-0.300369,-0.430754,-1.164265,0.293608,1,338,20.0,74.02
1,1.788302,-0.599212,-2.108530,0.105541,-0.419768,0.424503,0.306117,-1.297395,-0.365960,-0.193469,...,-1.120824,0.798896,-0.735984,-1.692172,-0.989390,1.180730,1,419,33.0,67.08
2,-0.024527,0.521169,-0.856823,-0.467126,-0.856623,0.542416,-1.578355,-0.025513,-1.748438,-0.108801,...,-0.789562,0.019656,-1.198619,1.141882,-0.438716,-0.378533,1,279,20.0,68.20
3,2.232087,0.245010,-0.081397,2.165611,-1.296768,0.386925,-1.609302,0.097405,-2.632485,0.009429,...,-0.900712,1.323254,-1.008070,-2.154635,1.147291,1.223645,1,524,68.0,53.34
4,-0.532192,-0.046631,-1.994325,0.103210,-1.187608,0.603498,0.119099,0.494659,-0.015966,1.025912,...,0.127415,-0.186732,-2.030141,-2.104558,0.112237,-0.022088,1,465,23.0,72.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,0.076996,1.060306,-0.374213,-0.734702,0.987911,3.004169,1.927672,0.183494,-2.541197,0.138758,...,0.343130,0.955492,0.042598,-0.262129,-1.845663,-2.069562,1,497,46.0,77.30
1779,0.526733,2.083623,-1.318357,0.365669,0.885012,1.430439,-0.444463,0.419993,-1.443482,0.104980,...,1.393086,1.244183,-1.869425,-1.892759,-1.115894,-0.072377,1,200,17.0,84.88
1780,0.258367,0.962929,1.196177,-0.475155,-1.180123,2.435375,-0.833579,0.755691,0.030383,1.244768,...,-0.655146,-0.589399,-1.204976,1.377089,0.634938,-0.676763,1,291,14.0,71.04
1781,0.092214,0.710328,0.870210,-0.486119,-0.559627,0.609152,0.108019,0.114833,-0.310847,0.426044,...,0.377715,0.458104,-0.948134,-0.174676,-0.203892,-0.439191,1,15,0.0,90.09
